In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, time

In [2]:
ccd = pd.read_csv('Consumer Complaints.csv')
ccd.head()

,Ticket ID,Ticket Created,Date of Issue,Time of Issue,Form,Method,Issue,Caller ID Number,Type of Call or Messge,Advertiser Business Number,City,State,Zip,Location (Center point of the Zip Code)
0,534,10/31/2014 12:29:24 PM +0000,NaN,NaN,Phone,Wireless (cell phone/other mobile device),Cramming (unauthorized charges on your phone b...,None,NaN,None,Minnetonka,MN,55345,"MN 55345\n(44.91531, -93.484053)"
1,535,10/31/2014 01:30:03 PM +0000,10/31/2014,07:03 am,Phone,Internet (VOIP),Telemarketing (including do not call and spoof...,978-957-4464,Live Voice,NaN,Berwick,PA,18603,"PA 18603\n(41.073476, -76.248784)"
2,537,10/31/2014 01:31:49 PM +0000,10/31/2014,12:36 AM,Phone,Wireless (cell phone/other mobile device),Telemarketing (including do not call and spoof...,786-309-9170,Live Voice,786-309-9170,Johnstown,PA,15902,"PA 15902\n(40.311892, -78.89308)"
3,538,10/31/2014 01:34:38 PM +0000,NaN,NaN,Phone,Wireless (cell phone/other mobile device),Availability (including rural call completion),NaN,NaN,NaN,Oconomowoc,WI,53066,"WI 53066\n(43.109168, -88.487198)"
4,542,10/31/2014 02:05:16 PM +0000,NaN,NaN,Phone,Internet (VOIP),Availability (including rural call completion),NaN,NaN,NaN,Somers,CT,06071,"CT 06071\n(41.994718, -72.449462)"


In [3]:
phone_df = ccd[ccd.Form == 'Phone']

# inefficient parsing incoming...
ti_s = phone_df['Time of Issue'].apply(lambda t: t.upper().replace(' ', '') if (type(t) == str) else t)

def fix_time_format(t):
    if type(t) == str:
        return (t[:t.index('A')] + 'AM') if ('A' in t) else (t[:t.index('P')] + 'PM')
    return t

# parse Time of Issue
ti_s = ti_s.apply(fix_time_format).apply(lambda t: datetime.strptime(t, '%I:%M%p').time() if type(t) == str else t).tolist()
#ti_s

In [4]:
# parse Date of Issue
di_s = phone_df['Date of Issue'].apply(lambda d: datetime.strptime(d, '%m/%d/%Y') if type(d) == str else d).tolist()

In [5]:
# parse Ticket Created
tc_s = phone_df['Ticket Created'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p +0000')).tolist()
#tc_s

In [6]:
di_s = [tc_s[i].date() if type(di_s[i]) != datetime else di_s[i].date() for i in range(len(di_s))]
ti_s = [tc_s[i].time() if type(ti_s[i]) == float else ti_s[i] for i in range(len(ti_s))]

In [7]:
# combine date and time
new_dt = [datetime.combine(d, t) for d,t in zip(di_s, ti_s)]

phone_df.insert(1, column='Issue Occurred', value=new_dt)
#phone_df.head()

In [8]:
# take out some unnecessary stuff, rearrange columns
new_cols = ['Issue Occurred', 'Form', 'Method', 'Issue', 'City', 'State', 'Zip']
phone_df = phone_df[new_cols]

# take out rows that have empty values for all location-based columns
phone_df = phone_df[phone_df.City.notna() & phone_df.State.notna() & phone_df.Zip.notna()]
#phone_df.head()

In [9]:
# fill missing values, capitalize "Time of Issue"
phone_df = phone_df.fillna('Unknown')